In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 7: Internet Protocol Version 7 ---</h2><p>While snooping around the local network of EBHQ, you compile a list of <a href="https://en.wikipedia.org/wiki/IP_address">IP addresses</a> (they're IPv7, of course; <a href="https://en.wikipedia.org/wiki/IPv6">IPv6</a> is much too limited). You'd like to figure out which IPs support <em>TLS</em> (transport-layer snooping).</p>
<p>An IP supports TLS if it has an Autonomous Bridge Bypass Annotation, or <span title="Any similarity to the pattern it describes is purely coincidental."><em>ABBA</em></span>.  An ABBA is any four-character sequence which consists of a pair of two different characters followed by the reverse of that pair, such as <code>xyyx</code> or <code>abba</code>.  However, the IP also must not have an ABBA within any hypernet sequences, which are contained by <em>square brackets</em>.</p>
<p>For example:</p>
<ul>
<li><code>abba[mnop]qrst</code> supports TLS (<code>abba</code> outside square brackets).</li>
<li><code>abcd[bddb]xyyx</code> does <em>not</em> support TLS (<code>bddb</code> is within square brackets, even though <code>xyyx</code> is outside square brackets).</li>
<li><code>aaaa[qwer]tyui</code> does <em>not</em> support TLS (<code>aaaa</code> is invalid; the interior characters must be different).</li>
<li><code>ioxxoj[asdfgh]zxcvbn</code> supports TLS (<code>oxxo</code> is outside square brackets, even though it's within a larger string).</li>
</ul>
<p><em>How many IPs</em> in your puzzle input support TLS?</p>
</article>


In [13]:
tests = [
    {
        "name": "supports TLS (abba outside square brackets)",
        "ip_adress": "abba[mnop]qrst",
        "expected": True,
    },
    {
        "name": "does not support TLS (bddb is within square brackets, even though xyyx is outside square brackets)",
        "ip_adress": "abcd[bddb]xyyx",
        "expected": False,
    },
    {
        "name": "does not support TLS (aaaa is invalid; the interior characters must be different)",
        "ip_adress": "aaaa[qwer]tyui",
        "expected": False,
    },
    {
        "name": "supports TLS (oxxo is outside square brackets, even though it's within a larger string).",
        "ip_adress": "ioxxoj[asdfgh]zxcvbn",
        "expected": True,
    },
    {
        "name": "multiple brackets",
        "ip_adress": "terznkuheuiksxrak[mvnbiknrfabvjwdkxn]cwddjxvgmetzjrkzea[xziqxlxbnvhkmqbos]fhxfhmqgpuadsubh[zeqlrmsxwvjemyw]nsfzmxgouassmcs",
        "expected": False,
    },
]


def supports_TLS(ip_adress: str) -> bool:
    abba_regex = re.compile(r"(.)(?!\1)(.)\2\1")
    inside_regex = re.compile(r"\[(.*?)\]")
    outside_regex = re.compile(r"(.*?)(?:\[.*?\]|$)")
    inside = " ".join(re.findall(inside_regex, ip_adress))
    outside = " ".join(re.findall(outside_regex, ip_adress))
    return (
        re.search(abba_regex, inside) is None
        and re.search(abba_regex, outside) is not None
    )


run_tests_params(supports_TLS, tests)


Test supports TLS (abba outside square brackets) passed, for supports_TLS.
Test does not support TLS (bddb is within square brackets, even though xyyx is outside square brackets) passed, for supports_TLS.
Test does not support TLS (aaaa is invalid; the interior characters must be different) passed, for supports_TLS.
Test supports TLS (oxxo is outside square brackets, even though it's within a larger string). passed, for supports_TLS.
Test multiple brackets passed, for supports_TLS.
Success


In [14]:
adresses = """
abba[mnop]qrst
abcd[bddb]xyyx
aaaa[qwer]tyui
ioxxoj[asdfgh]zxcvbn
"""


def count_TLS_support(adresses: str) -> int:
    return sum(1 for l in adresses.strip().splitlines() if supports_TLS(l))


count_TLS_support(adresses)

2

In [15]:
with open("../input/day7.txt") as f:
    print("Part I:", count_TLS_support(f.read()))

Part I: 110


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>110</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You would also like to know which IPs support <em>SSL</em> (super-secret listening).</p>
<p>An IP supports SSL if it has an Area-Broadcast Accessor, or <em>ABA</em>, anywhere in the supernet sequences (outside any square bracketed sections), and a corresponding Byte Allocation Block, or <em>BAB</em>, anywhere in the hypernet sequences. An ABA is any three-character sequence which consists of the same character twice with a different character between them, such as <code>xyx</code> or <code>aba</code>. A corresponding BAB is the same characters but in reversed positions: <code>yxy</code> and <code>bab</code>, respectively.</p>
<p>For example:</p>
<ul>
<li><code>aba[bab]xyz</code> supports SSL (<code>aba</code> outside square brackets with corresponding <code>bab</code> within square brackets).</li>
<li><code>xyx[xyx]xyx</code> does <em>not</em> support SSL (<code>xyx</code>, but no corresponding <code>yxy</code>).</li>
<li><code>aaa[kek]eke</code> supports SSL (<code>eke</code> in supernet with corresponding <code>kek</code> in hypernet; the <code>aaa</code> sequence is not related, because the interior character must be different).</li>
<li><code>zazbz[bzb]cdb</code> supports SSL (<code>zaz</code> has no corresponding <code>aza</code>, but <code>zbz</code> has a corresponding <code>bzb</code>, even though <code>zaz</code> and <code>zbz</code> overlap).</li>
</ul>
<p><em>How many IPs</em> in your puzzle input support SSL?</p>
</article>

</main>


In [23]:
tests = [
    {
        "name": "supports SSL (aba outside square brackets with corresponding bab within square brackets).",
        "ip_adress": "aba[bab]xyz",
        "expected": True,
    },
    {
        "name": "does not support SSL (xyx, but no corresponding yxy).",
        "ip_adress": "xyx[xyx]xyx",
        "expected": False,
    },
    {
        "name": "supports SSL (eke in supernet with corresponding kek in hypernet; the aaa sequence is not related, because the interior character must be different).",
        "ip_adress": "aaa[kek]eke",
        "expected": True,
    },
    {
        "name": "supports SSL (zaz has no corresponding aza, but zbz has a corresponding bzb, even though zaz and zbz overlap).",
        "ip_adress": "zazbz[bzb]cdb",
        "expected": True,
    },
]


def supports_SSL(ip_adress: str) -> bool:
    inside_regex = re.compile(r"\[(.*?)\]")
    outside_regex = re.compile(r"(.*?)(?:\[.*?\]|$)")
    inside = " ".join(re.findall(inside_regex, ip_adress))
    outside = " ".join(re.findall(outside_regex, ip_adress))
    for i in range(2, len(outside)):
        if outside[i - 2] == outside[i] and outside[i - 1] != outside[i]:
            sbs = outside[i - 1] + outside[i] + outside[i - 1]
            if sbs in inside:
                return True
    return False


run_tests_params(supports_SSL, tests)


Test supports SSL (aba outside square brackets with corresponding bab within square brackets). passed, for supports_SSL.
Test does not support SSL (xyx, but no corresponding yxy). passed, for supports_SSL.
Test supports SSL (eke in supernet with corresponding kek in hypernet; the aaa sequence is not related, because the interior character must be different). passed, for supports_SSL.
Test supports SSL (zaz has no corresponding aza, but zbz has a corresponding bzb, even though zaz and zbz overlap). passed, for supports_SSL.
Success


In [25]:
adresses = """
aba[bab]xyz
xyx[xyx]xyx
aaa[kek]eke
zazbz[bzb]cdb
"""


def count_SSL_support(adresses: str) -> int:
    return sum(1 for l in adresses.strip().splitlines() if supports_SSL(l))


count_SSL_support(adresses)

3

In [26]:
with open("../input/day7.txt") as f:
    print("Part II:", count_SSL_support(f.read()))

Part II: 242


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>242</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="7/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Internet+Protocol+Version+7%22+%2D+Day+7+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F7&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Internet+Protocol+Version+7%22+%2D+Day+7+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F7';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
